#  Usage/Benchmark Tests
-------------------


In [1]:
# Load Params 
using BenchmarkTools

push!(LOAD_PATH, "$(pwd())/src"); using DarkWingDuck;
const dwd = DarkWingDuck

┌ Info: Precompiling DarkWingDuck [top-level]
└ @ Base loading.jl:1260


DarkWingDuck

In [2]:
"""Generate uniform random points along a surface"""
function genRandomCoords(x0::Float64, x1::Float64, y0::Float64, y1::Float64, n::Int)::Array{dwd.Coord}
    X = ((ones(n) * x0) + (rand(n) * (x1 - x0))) 
    Y = ((ones(n) * y0) + (rand(n) * (y1 - y0)))
    return [dwd.Coord(x, y) for (x,y) in zip(X,Y)]
end

genRandomCoords

In [6]:
const QT_TEST_SIZE, QT_TEST_RM_SIZE = 500_000, 100_000

## Inserts - Tree Build
ps = genRandomCoords(0., 1., 0., 1., QT_TEST_SIZE);

#TODO: constructor should take anyFloat
sampleQTree = dwd.qtBox(dwd.Coord(.0, .0), 1.)

@time for p in ps 
    dwd.insertIntoQuadTree!(sampleQTree, p)
end

  0.830382 seconds (11.43 M allocations: 368.550 MiB, 12.88% gc time)


In [4]:
@benchmark dwd.queryRange(sampleQTree, dwd.Box(dwd.Coord(rand() , rand()), rand()/2))

BenchmarkTools.Trial: 
  memory estimate:  4.09 KiB
  allocs estimate:  68
  --------------
  minimum time:     3.092 μs (0.00% GC)
  median time:      76.726 μs (0.00% GC)
  mean time:        129.462 μs (5.35% GC)
  maximum time:     2.483 ms (84.88% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [4]:
#Test in kilometers, 5km ~0.045 on a flat plane
@benchmark DarkWingDuck.radialSearch(sampleQTree, dwd.Coord(rand(), rand()), Float32(5.))

BenchmarkTools.Trial: 
  memory estimate:  160.27 MiB
  allocs estimate:  470183
  --------------
  minimum time:     111.309 ms (9.51% GC)
  median time:      116.132 ms (11.37% GC)
  mean time:        131.018 ms (20.85% GC)
  maximum time:     268.249 ms (59.76% GC)
  --------------
  samples:          39
  evals/sample:     1

In [7]:
# Oyy, Benchmarking function!() is pretty expensive, but passing Modify == false is ~3-4x quicker,
removeCoords = ps[map(x -> Int64(floor(x + 1)), rand(QT_TEST_RM_SIZE) * QT_TEST_RM_SIZE)]
@benchmark (for rmc in removeCoords dwd.removePointRebuild!(sampleQTree, rmc, true) end)